In [2]:
import sys,string,random
import subprocess

In [4]:
def computeCountAndLists(s):
  #WARNING: Use of function count(s,'UU') returns 1 on word UUU
  #since it apparently counts only nonoverlapping words UU
  #For this reason, we work with the indices.

  #Initialize lists and mono- and dinucleotide dictionaries
  List = {} #List is a dictionary of lists
  List['A'] = []
  List['C'] = []
  List['G'] = []
  List['T'] = []
  List['N'] = []

  nuclList   = ["A","C","G","T","N"]
  s = s.upper()
  s = s.replace("T","T")
  nuclCnt = {}  #empty dictionary
  dinuclCnt = {}  #empty dictionary
  for x in nuclList:
    nuclCnt[x]=0
    dinuclCnt[x]={}
    for y in nuclList:
      dinuclCnt[x][y]=0

  #Compute count and lists
  nuclCnt[s[0]] = 1
  nuclTotal     = 1
  dinuclTotal   = 0
  for i in range(len(s)-1):
    x = s[i]; y = s[i+1]
    List[x].append( y )
    nuclCnt[y] += 1; nuclTotal  += 1
    dinuclCnt[x][y] += 1; dinuclTotal += 1
  assert (nuclTotal==len(s))
  assert (dinuclTotal==len(s)-1)
  return nuclCnt,dinuclCnt,List
 
 
def chooseEdge(x,dinuclCnt):
  numInList = 0
  for y in ['A','C','G','T','N']:
    numInList += dinuclCnt[x][y]
  z = random.random()
  denom=dinuclCnt[x]['A']+dinuclCnt[x]['C']+dinuclCnt[x]['G']+dinuclCnt[x]['T']+dinuclCnt[x]['N']
  numerator = dinuclCnt[x]['A']
  if z < float(numerator)/float(denom):
    dinuclCnt[x]['A'] -= 1
    return 'A'
  numerator += dinuclCnt[x]['C']
  if z < float(numerator)/float(denom):
    dinuclCnt[x]['C'] -= 1
    return 'C'
  numerator += dinuclCnt[x]['N']
  if z < float(numerator)/float(denom):
    dinuclCnt[x]['N'] -= 1
    return 'N'
  numerator += dinuclCnt[x]['G']
  if z < float(numerator)/float(denom):
    dinuclCnt[x]['G'] -= 1
    return 'G'
  dinuclCnt[x]['T'] -= 1
  return 'T'


def connectedToLast(edgeList,nuclList,lastCh):
  D = {}
  for x in nuclList: D[x]=0
  for edge in edgeList:
    a = edge[0]; b = edge[1]
    if b==lastCh: D[a]=1
  for i in range(2):
    for edge in edgeList:
      a = edge[0]; b = edge[1]
      if D[b]==1: D[a]=1
  ok = 0
  for x in nuclList:
    if x!=lastCh and D[x]==0: return 0
  return 1
 

def eulerian(s):
  nuclCnt,dinuclCnt,List = computeCountAndLists(s)
  #compute nucleotides appearing in s
  nuclList = []
  for x in ["A","C","G","T","N"]:
    if x in s: nuclList.append(x)
  #compute numInList[x] = number of dinucleotides beginning with x
  numInList = {}
  for x in nuclList:
    numInList[x]=0
    for y in nuclList:
      numInList[x] += dinuclCnt[x][y]
  #create dinucleotide shuffle L 
  firstCh = s[0]  #start with first letter of s
  lastCh  = s[-1]
  edgeList = []
  for x in nuclList:
    if x!= lastCh: edgeList.append( [x,chooseEdge(x,dinuclCnt)] )
  ok = connectedToLast(edgeList,nuclList,lastCh)
  return ok,edgeList,nuclList,lastCh


def shuffleEdgeList(L):
  n = len(L); barrier = n
  for i in range(n-1):
    z = int(random.random() * barrier)
    tmp = L[z]
    L[z]= L[barrier-1]
    L[barrier-1] = tmp
    barrier -= 1
  return L


def dinuclShuffle(s):
  ok = 0
  while not ok:
    ok,edgeList,nuclList,lastCh = eulerian(s)
  nuclCnt,dinuclCnt,List = computeCountAndLists(s)

  #remove last edges from each vertex list, shuffle, then add back
  #the removed edges at end of vertex lists.
  for [x,y] in edgeList: List[x].remove(y)
  for x in nuclList: shuffleEdgeList(List[x])
  for [x,y] in edgeList: List[x].append(y)

  #construct the eulerian path
  L = [s[0]]; prevCh = s[0]
  for i in range(len(s)-2):
    ch = List[prevCh][0] 
    L.append( ch )
    del List[prevCh][0]
    prevCh = ch
  L.append(s[-1])
  t = "".join(L)
  return t


In [6]:
# Generate 10,000 random DNA strings, shuffle them, and compare dinucleotide frequencies for each sequence

def random_dna_string(length):
    return ''.join(random.choices('ACGT', k=length))

num_sequences = 10000
seq_length = 100  # You can adjust the length as needed

matches = 0

for _ in range(num_sequences):
    seq = random_dna_string(seq_length)
    shuffled_seq = dinuclShuffle(seq)
    # Count dinucleotides in original
    orig_freq = {}
    shuf_freq = {}
    for x in 'ACGT':
        for y in 'ACGT':
            orig_freq[x+y] = 0
            shuf_freq[x+y] = 0
    for i in range(len(seq)-1):
        orig_freq[seq[i:i+2]] += 1
        shuf_freq[shuffled_seq[i:i+2]] += 1
    if orig_freq == shuf_freq:
        matches += 1

print(f"Number of sequences where dinucleotide frequencies match: {matches} out of {num_sequences}")


Number of sequences where dinucleotide frequencies match: 10000 out of 10000


In [7]:
seq

'TTGAGCCGGTGTCAGGCTCTCTCATTAGTTATAGGGGAATTGCGGGCAGGATTTCTATAGCATAATCATTATCTCTCATCGACGGAGTATGGGAAGCGAG'

In [8]:
shuffled_seq

'TATTCATGAACTCATTATGCTTATAGGAGATCTTCTGGATCTAGCGGTCGTTGGCAGAAAGGGGATAGCATAGGGTTCTAGTCATCAGCGATTCGGGCCG'

In [9]:
orig_freq

{'AA': 3,
 'AC': 1,
 'AG': 9,
 'AT': 11,
 'CA': 6,
 'CC': 1,
 'CG': 5,
 'CT': 6,
 'GA': 7,
 'GC': 6,
 'GG': 11,
 'GT': 4,
 'TA': 8,
 'TC': 10,
 'TG': 4,
 'TT': 7}

In [10]:
shuf_freq

{'AA': 3,
 'AC': 1,
 'AG': 9,
 'AT': 11,
 'CA': 6,
 'CC': 1,
 'CG': 5,
 'CT': 6,
 'GA': 7,
 'GC': 6,
 'GG': 11,
 'GT': 4,
 'TA': 8,
 'TC': 10,
 'TG': 4,
 'TT': 7}